## Import Libraries

In [2]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
import transformers
from tqdm import tqdm, trange
from ast import literal_eval

     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 45.5 MB/s 
     |████████████████████████████████| 6.5 MB 49.7 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/LAP/Subjects/AP1/labs

/content/drive/MyDrive/LAP/Subjects/AP1/labs


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

## Load and Preprocess Training Data

Dataset will be tokenized then split into training and validation sets. The validation set will be used to monitor training. For testing a separate test set will be loaded for analysis.

In [ ]:
train_set = "../datasets/absa2016/en-train-acd-multilabel-transformers.csv"

In [ ]:
# the file might have to be called train.csv
df = pd.read_csv(train_set)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1708 non-null   int64 
 1   comment_text              1708 non-null   object
 2   AMBIENCE#GENERAL          1708 non-null   int64 
 3   DRINKS#PRICES             1708 non-null   int64 
 4   DRINKS#QUALITY            1708 non-null   int64 
 5   DRINKS#STYLE_OPTIONS      1708 non-null   int64 
 6   FOOD#PRICES               1708 non-null   int64 
 7   FOOD#QUALITY              1708 non-null   int64 
 8   FOOD#STYLE_OPTIONS        1708 non-null   int64 
 9   LOCATION#GENERAL          1708 non-null   int64 
 10  RESTAURANT#GENERAL        1708 non-null   int64 
 11  RESTAURANT#MISCELLANEOUS  1708 non-null   int64 
 12  RESTAURANT#PRICES         1708 non-null   int64 
 13  SERVICE#GENERAL           1708 non-null   int64 
dtypes: int64(13), object(1)


,id,comment_text,AMBIENCE#GENERAL,DRINKS#PRICES,DRINKS#QUALITY,DRINKS#STYLE_OPTIONS,FOOD#PRICES,FOOD#QUALITY,FOOD#STYLE_OPTIONS,LOCATION#GENERAL,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,SERVICE#GENERAL
0,2202,Judging from previous posts this used to be a ...,0,0,0,0,0,0,0,0,1,0,0,0
1,9326,"We, there were four of us, arrived at noon - t...",0,0,0,0,0,0,0,0,0,0,0,1
2,1034,"They never brought us complimentary noodles, i...",0,0,0,0,0,0,0,0,0,0,0,1
3,4180,The food was lousy - too sweet or too salty an...,0,0,0,0,0,1,1,0,0,0,0,0
4,1932,"After all that, they complained to me about th...",0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
print('Unique comments: ', df.comment_text.nunique() == df.shape[0])
print('Null values: ', df.isnull().values.any())
# df[df.isna().any(axis=1)]

Unique comments:  False
Null values:  False


In [ ]:
print('average sentence length: ', df.comment_text.str.split().str.len().mean())
print('stdev sentence length: ', df.comment_text.str.split().str.len().std())

average sentence length:  12.507611241217798
stdev sentence length:  8.285011666209963


In [ ]:
cols = df.columns
label_cols = list(cols[2:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE_OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']


In [ ]:
print('Count of 1 per label: \n', df[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df[label_cols].eq(0).sum())

Count of 1 per label: 
 AMBIENCE#GENERAL            226
DRINKS#PRICES                20
DRINKS#QUALITY               46
DRINKS#STYLE_OPTIONS         30
FOOD#PRICES                  82
FOOD#QUALITY                681
FOOD#STYLE_OPTIONS          128
LOCATION#GENERAL             28
RESTAURANT#GENERAL          421
RESTAURANT#MISCELLANEOUS     97
RESTAURANT#PRICES            80
SERVICE#GENERAL             419
dtype: int64 

Count of 0 per label: 
 AMBIENCE#GENERAL            1482
DRINKS#PRICES               1688
DRINKS#QUALITY              1662
DRINKS#STYLE_OPTIONS        1678
FOOD#PRICES                 1626
FOOD#QUALITY                1027
FOOD#STYLE_OPTIONS          1580
LOCATION#GENERAL            1680
RESTAURANT#GENERAL          1287
RESTAURANT#MISCELLANEOUS    1611
RESTAURANT#PRICES           1628
SERVICE#GENERAL             1289
dtype: int64


In [ ]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows

## ASSIGNMENT 1 

+ TODO: Generate an extra column in the pandas dataframe containing:
++ one_hot_labels as header.
++ the list of aspect values extracted from each aspect column.

The dataframe obtained should be as follows:

In [ ]:
## TODO generate one_hot_labels column
one_hot_labels = []
for index, row in df.iterrows():
    one_hot = []
    for label in label_cols:
        one_hot.append(row[label])
    one_hot_labels.append(one_hot)

df["one_hot_labels"] = one_hot_labels
df

,id,comment_text,AMBIENCE#GENERAL,DRINKS#PRICES,DRINKS#QUALITY,DRINKS#STYLE_OPTIONS,FOOD#PRICES,FOOD#QUALITY,FOOD#STYLE_OPTIONS,LOCATION#GENERAL,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,SERVICE#GENERAL,one_hot_labels
0,329,"What really makes it shine is the food, which ...",0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
1,8945,"Nevertheless, I finished my plate, and that's ...",0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
2,1689,"Each time we've been, the front of house staff...",0,0,0,0,0,1,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]"
3,1364,"those rolls were big, but not good and sashimi...",0,0,0,0,0,1,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"
4,3366,And it was quick which is very important.,0,0,0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,4180,The food was lousy - too sweet or too salty an...,0,0,0,0,0,1,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"
1704,7570,The only thing that strikes you is the decor?(...,1,0,0,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1705,7366,"Excellent atmosphere, delicious dishes good an...",1,0,0,0,0,1,0,0,0,0,0,1,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]"
1706,9180,The food was exceptional.,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [ ]:
labels = list(df.one_hot_labels.values)
comments = list(df.comment_text.values)

Load the pretrained tokenizer that corresponds to your choice in model. e.g.,

```
BERT:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) 


RoBERTa:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False)
```


NOTE: In order to avoid memory issues with Google Colab, I enforce a max_length of 100 tokens. Note that some sentences may not adequately represent each label because of this.

## ASSIGNMENT 2 

+ TODO: Instantiate the tokenizer from "bert-base-uncased" model in lowercase mode. HINT: Check huggingface course on tokenizers.
+ TODO: Investigate how defining different max_lengths affect performance on the test set evaluation. You may try values of 64, 128 (in addition to 100).


In [ ]:
# TODO instantiate tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_length = 100
encodings = tokenizer.batch_encode_plus(comments, truncation=True, max_length=max_length, pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [ ]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [1631, 1625, 1618, 1607, 1513, 1439, 1401, 1355, 1271, 1212, 1179, 1164, 1132, 1112, 1044, 1013, 999, 928, 881, 871, 818, 793, 786, 781, 760, 727, 534, 529, 517, 463, 456, 453, 406, 383, 382, 284, 262, 205, 194, 72, 53, 35, 29, 24, 5]


In [ ]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

Be sure to handle all classes during validation using "stratify" during train/validation split:

In [ ]:
# Use train_test_split to split our data into train and validation sets

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.10, stratify = labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

In [ ]:
# Select a batch size for training.
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
torch.save(validation_dataloader,'../resources/bert-multilabel-acd-en/validation_data_loader.pt')
torch.save(train_dataloader,'../resources/bert-multilabel-acd-en/train_data_loader.pt')

## Load Model & Set Params

Load the appropriate model below, each model already contains a single dense layer for classification on top.



```
BERT:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

RoBERTa:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
```



## ASSIGNMENT 3

+ TODO: load the model for SequenceClassification corresponding to the tokenizer instantiated above.

In [ ]:
# TODO Load model, the pretrained model will include a single linear classification layer on top for classification.
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Setting custom optimization parameters for the AdamW optimizer https://huggingface.co/transformers/main_classes/optimizer_schedules.html

In [ ]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
#pip install tensorflow_addons

In [ ]:
#import tensorflow as tf
#import tensorflow_addons as tfa

#optimizer = tfa.optimizers.AdamW(optimizer_grouped_parameters,lr=2e-5)
#optimizer = tfa.optimizers.AdamW(model.parameters(),lr=2e-5)  # Default optimization
optimizer = torch.optim.AdamW(optimizer_grouped_parameters,lr=2e-5)

## Train Model

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('F1 Macro Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.4186136132663058


Epoch:  25%|██▌       | 1/4 [00:53<02:39, 53.15s/it]

F1 Validation Accuracy:  41.00719424460432
F1 Macro Validation Accuracy:  22.15568862275449
Train loss: 0.2546456815655698


Epoch:  50%|█████     | 2/4 [01:46<01:46, 53.12s/it]

F1 Validation Accuracy:  44.52296819787986
F1 Macro Validation Accuracy:  24.550898203592812
Train loss: 0.20687272161552586


Epoch:  75%|███████▌  | 3/4 [02:39<00:53, 53.04s/it]

F1 Validation Accuracy:  69.85915492957746
F1 Macro Validation Accuracy:  53.293413173652695
Train loss: 0.16188461371918314


Epoch: 100%|██████████| 4/4 [03:32<00:00, 53.15s/it]

F1 Validation Accuracy:  74.05405405405406
F1 Macro Validation Accuracy:  59.2814371257485


In [ ]:
torch.save(model.state_dict(), '../resources/bert-multilabel-acd-en/bert-multilable-acd-en.pt')

## Load and Preprocess Test Data

In [ ]:
test_set = "../datasets/absa2016/en-test-acd-multilabel-transformers.csv"

In [ ]:
test_df = pd.read_csv(test_set)
test_df.head()

,id,comment_text,AMBIENCE#GENERAL,DRINKS#PRICES,DRINKS#QUALITY,DRINKS#STYLE_OPTIONS,FOOD#PRICES,FOOD#QUALITY,FOOD#STYLE_OPTIONS,LOCATION#GENERAL,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,SERVICE#GENERAL
0,12201,Yum!,0,0,0,0,0,1,0,0,0,0,0,0
1,19325,Serves really good sushi.,0,0,0,0,0,1,0,0,0,0,0,0
2,11033,Not the biggest portions but adequate.,0,0,0,0,0,0,1,0,0,0,0,0
3,14179,Green Tea creme brulee is a must!,0,0,0,0,0,1,0,0,0,0,0,0
4,11931,Don't leave the restaurant without it.,0,0,0,0,0,1,0,0,0,0,0,0


## ASSIGNMENT 4

+ TODO add one_hot_labels column to test data as for ASSIGNMENT 1.

In [ ]:
cols_test = test_df.columns
label_cols_test = list(cols_test[2:])
num_labels_test = len(label_cols_test)
# TODO add one_hot_labels column to test set
one_hot_labels = []
for index, row in test_df.iterrows():
    one_hot = []
    for label in label_cols_test:
        one_hot.append(row[label])
    one_hot_labels.append(one_hot)

test_df["one_hot_labels"] = one_hot_labels
test_df.head()

,id,comment_text,AMBIENCE#GENERAL,DRINKS#PRICES,DRINKS#QUALITY,DRINKS#STYLE_OPTIONS,FOOD#PRICES,FOOD#QUALITY,FOOD#STYLE_OPTIONS,LOCATION#GENERAL,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,SERVICE#GENERAL,one_hot_labels
0,12201,Yum!,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
1,19325,Serves really good sushi.,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
2,11033,Not the biggest portions but adequate.,0,0,0,0,0,0,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,14179,Green Tea creme brulee is a must!,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,11931,Don't leave the restaurant without it.,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [ ]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.comment_text.values)

In [ ]:
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,truncation=True, max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'../resources/bert-multilabel-acd-en/test_data_loader.pt')

## Prediction and Evaluation

In [ ]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

We need to threshold our sigmoid function outputs which range from [0, 1]. Below I use 0.50 as a threshold.

## ASSIGNMENT 5

+ TODO use scikit-learn functions to calculate F1 micro and Accuracy scores. HINT: you need to use true_bools and pred_bools from above.
+ TODO: use scikit-learn function to provide a classification report.

Output should be similar to the following:

In [ ]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# TODO Print and save classification report
from sklearn.metrics import classification_report
clf_report = classification_report(true_bools, pred_bools, target_names=label_cols_test, output_dict=True)
clf_report_df = pd.DataFrame(clf_report).transpose()
clf_report_df.to_csv("../resources/bert-multilabel-acd-en/clf_report.csv")
clf_report_df

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
AMBIENCE#GENERAL,0.791667,0.666667,0.723810,57.0
DRINKS#PRICES,0.000000,0.000000,0.000000,3.0
DRINKS#QUALITY,0.000000,0.000000,0.000000,21.0
DRINKS#STYLE_OPTIONS,0.000000,0.000000,0.000000,12.0
FOOD#PRICES,1.000000,0.136364,0.240000,22.0
FOOD#QUALITY,0.872881,0.911504,0.891775,226.0
FOOD#STYLE_OPTIONS,1.000000,0.020833,0.040816,48.0
LOCATION#GENERAL,0.000000,0.000000,0.000000,13.0
RESTAURANT#GENERAL,0.819672,0.704225,0.757576,142.0
RESTAURANT#MISCELLANEOUS,0.000000,0.000000,0.000000,33.0


## Output Dataframe

In [ ]:
idx2label = dict(zip(range(12),label_cols))
print(idx2label)

{0: 'AMBIENCE#GENERAL', 1: 'DRINKS#PRICES', 2: 'DRINKS#QUALITY', 3: 'DRINKS#STYLE_OPTIONS', 4: 'FOOD#PRICES', 5: 'FOOD#QUALITY', 6: 'FOOD#STYLE_OPTIONS', 7: 'LOCATION#GENERAL', 8: 'RESTAURANT#GENERAL', 9: 'RESTAURANT#MISCELLANEOUS', 10: 'RESTAURANT#PRICES', 11: 'SERVICE#GENERAL'}


In [ ]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [ ]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

# BONUS ASSIGNMENT 6

In this assignment we will decode the input ids from the tokenized texts using the tokenizer instantiated above and will use them to generate a dataframe in which to add the text of the review, the true labels and the predicted labels. We will then save this dataframe to a csv which could be used to manually inspect the predictions of the model with respect to the gold standard.

+ TODO: decode the texts.
+ TODO: create a dataframe containing three columns: the texts, the true labels and the predicted labels.
+ TODO: save it into a csv.

The result should be something like the following:

In [ ]:
# TODO Decoding input ids to comment text
comments_test = tokenizer.batch_decode(tokenized_texts, skip_special_tokens=True)

In [ ]:
# TODO Converting lists to df
comparisons_df = pd.DataFrame({"comment_text": comments_test, "true_labels": true_label_texts, "pred_label": pred_label_texts})
comparisons_df.to_csv("../resources/bert-multilabel-acd-en/comparisons.csv")
comparisons_df.head()

,comment_text,true_labels,pred_label
0,yum!,[FOOD#QUALITY],[RESTAURANT#GENERAL]
1,serves really good sushi.,[FOOD#QUALITY],[FOOD#QUALITY]
2,not the biggest portions but adequate.,[FOOD#STYLE_OPTIONS],[FOOD#QUALITY]
3,green tea creme brulee is a must!,[FOOD#QUALITY],[FOOD#QUALITY]
4,don't leave the restaurant without it.,[FOOD#QUALITY],[RESTAURANT#GENERAL]


# BONUS ASSIGNMENT 7

+ TODO: Can you generate the required data for multilabel aspect category detection using the "acb" datasets available for other languages?

In [114]:
def get_label(label):
    return label[9:]

def transform_df(df):
    # change labels to a list
    labels_flair = list(map(str.split, df["labels"]))
    df["labels"] = labels_flair

    # remove flair label prefix
    labels = [list(map(get_label, labels)) for labels in labels_flair]
    df["labels"] = labels

    # get dataframe columns
    flat_list = [item for sublist in labels for item in sublist]
    cols = sorted(set(flat_list))
    cols.remove("")

    # set values to 0 by default
    for col in cols:
        df[col] = 0

    # set values to 1
    for index, row in df.iterrows():
        labels = row["labels"]
        for label in labels:
            if label != "":
                df.at[index, label] = 1
    
    # drop labels column
    df = df.drop('labels', axis=1)
    return df

In [122]:
df_train_es = pd.read_csv("../datasets/absa2016/es-train-acb.tsv", delimiter="\t", 
                          header=None, names=["labels", "comment_text"])
df_train_es.head()

,labels,comment_text
0,__label__RESTAURANT#GENERAL,Nos sentimos muy a gusto.
1,__label__SERVICE#GENERAL __label__AMBIENCE#GEN...,"Buen servicio, ambiente Acogedor y tranquilo,..."
2,__label__RESTAURANT#GENERAL,Muy recomendable
3,__label__SERVICE#GENERAL __label__SERVICE#GENERAL,"La atencion es muy buena, los camareros estan ..."
4,__label__FOOD#QUALITY,"Y la comida espectacular, por eso lo bueno se ..."


In [123]:
df_train_es = transform_df(df_train_es)
df_train_es.to_csv("../datasets/absa2016/es-train-acd-multilabel-transformers.csv")
df_train_es.head()

,comment_text,AMBIENCE#GENERAL,DRINKS#PRICES,DRINKS#QUALITY,DRINKS#STYLE_OPTIONS,FOOD#PRICES,FOOD#QUALITY,FOOD#STYLE_OPTIONS,LOCATION#GENERAL,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,SERVICE#GENERAL
0,Nos sentimos muy a gusto.,0,0,0,0,0,0,0,0,1,0,0,0
1,"Buen servicio, ambiente Acogedor y tranquilo,...",1,0,0,0,0,1,0,0,0,0,0,1
2,Muy recomendable,0,0,0,0,0,0,0,0,1,0,0,0
3,"La atencion es muy buena, los camareros estan ...",0,0,0,0,0,0,0,0,0,0,0,1
4,"Y la comida espectacular, por eso lo bueno se ...",0,0,0,0,0,1,0,0,0,0,0,0


In [124]:
df_test_es = pd.read_csv("../datasets/absa2016/es-test-acb.tsv", delimiter="\t", 
                          header=None, names=["labels", "comment_text"])
df_test_es.head()

,labels,comment_text
0,__label__FOOD#QUALITY,La comida estuvo muy sabrosa.
1,__label__FOOD#STYLE_OPTIONS __label__FOOD#STYL...,Quien sea amante de la carne tiene una carta b...
2,__label__SERVICE#GENERAL,Lo único que nos sorprendió es que nos sirvier...
3,-,"Buff,"
4,-,No se donde empezar!


In [125]:
df_test_es = transform_df(df_test_es)
df_test_es.to_csv("../datasets/absa2016/es-test-acd-multilabel-transformers.csv")
df_test_es.head()

,comment_text,AMBIENCE#GENERAL,DRINKS#PRICES,DRINKS#QUALITY,DRINKS#STYLE_OPTIONS,FOOD#PRICES,FOOD#QUALITY,FOOD#STYLE_OPTIONS,LOCATION#GENERAL,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,SERVICE#GENERAL
0,La comida estuvo muy sabrosa.,0,0,0,0,0,1,0,0,0,0,0,0
1,Quien sea amante de la carne tiene una carta b...,0,0,0,0,0,0,1,0,0,0,0,0
2,Lo único que nos sorprendió es que nos sirvier...,0,0,0,0,0,0,0,0,0,0,0,1
3,"Buff,",0,0,0,0,0,0,0,0,0,0,0,0
4,No se donde empezar!,0,0,0,0,0,0,0,0,0,0,0,0
